This file is produced by Hengyi Li and Jingbo Wang

2023.3.10

# Fundamentals of machine learning

## Generalization: The goal of machine learning

We can setup our envrinment by importing those libraries 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import imdb

2023-03-10 10:38:23.696959: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Before start, we can check our running environment by running those code

In [2]:
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_physical_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Improving generalization

To improve the generalization, we have several ways to do it. 

* Dataset curation

* Feature engineering

* Using early stopping

* Reducing the network's size

  * Reducing the network's size
  * Adding L2 weight regularization to the model
  * Adding dropout to the IMDB model

### Dataset curation

* Make sure you have enough data. Remember that you need a _**dense sampling**_ of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Make sure you have enough data. Remember that you need a dense sampling of the input-cross-output space. More data will yield a better model. Sometimes, problems that seem impossible at first become solvable with a larger dataset.
* Clean your data and deal with missing values.
* If you have many features and you aren’t sure which ones are actually useful, do feature selection.


So the num_words restricted the top 10,000 most frequently occurring words in the training data. Rare words will be discarded. And we can do better than that by increasing the num_words to 25000 

**Original Model**

In [3]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 4s 91ms/step - loss: 0.5671 - accuracy: 0.7546 - val_loss: 0.4352 - val_accuracy: 0.8585
Epoch 2/20
30/30 [==============================] - 1s 18ms/step - loss: 0.3557 - accuracy: 0.8850 - val_loss: 0.3264 - val_accuracy: 0.8840
Epoch 3/20
30/30 [==============================] - 0s 16ms/step - loss: 0.2627 - accuracy: 0.9141 - val_loss: 0.2884 - val_accuracy: 0.8898
Epoch 4/20
30/30 [==============================] - 0s 16ms/step - loss: 0.2079 - accuracy: 0.9299 - val_loss: 0.2751 - val_accuracy: 0.8900
Epoch 5/20
30/30 [==============================] - 0s 17ms/step - loss: 0.1716 - accuracy: 0.9439 - val_loss: 0.2790 - val_accuracy: 0.8882
Epoch 6/20
30/30 [==============================] - 0s 16ms/step - loss: 0.1469 - accuracy: 0.9536 - val_loss: 0.2948 - val_accuracy: 0.8833
Epoch 7/20
30/30 [==============================] - 0s 16ms/step - loss: 0.1273 - accuracy: 0.9607 - val_loss: 0.2927 - val_accuracy: 0.8870
Epoch 8/20
30

**Practice Model**

Here, we change the num_words to 25000 

In [4]:
(train_data, train_labels), _ = imdb.load_data(num_words=25000)

def vectorize_sequences(sequences, dimension=25000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 8s 227ms/step - loss: 0.5399 - accuracy: 0.7689 - val_loss: 0.4050 - val_accuracy: 0.8743
Epoch 2/20
30/30 [==============================] - 2s 61ms/step - loss: 0.3186 - accuracy: 0.9041 - val_loss: 0.3058 - val_accuracy: 0.8922
Epoch 3/20
30/30 [==============================] - 1s 47ms/step - loss: 0.2228 - accuracy: 0.9326 - val_loss: 0.2987 - val_accuracy: 0.8797
Epoch 4/20
30/30 [==============================] - 1s 33ms/step - loss: 0.1670 - accuracy: 0.9489 - val_loss: 0.2822 - val_accuracy: 0.8868
Epoch 5/20
30/30 [==============================] - 1s 32ms/step - loss: 0.1286 - accuracy: 0.9638 - val_loss: 0.3127 - val_accuracy: 0.8777
Epoch 6/20
30/30 [==============================] - 1s 33ms/step - loss: 0.1038 - accuracy: 0.9722 - val_loss: 0.2835 - val_accuracy: 0.8898
Epoch 7/20
30/30 [==============================] - 1s 34ms/step - loss: 0.0801 - accuracy: 0.9813 - val_loss: 0.3051 - val_accuracy: 0.8877
Epoch 8/20
3

As it shown above, after change the num_words and the batch size, make a bigger dataset and more training on each epoch, the accuracy gets visible improvement

### Feature engineering

**Original Model**

In [5]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 3s 64ms/step - loss: 0.5585 - accuracy: 0.7769 - val_loss: 0.4398 - val_accuracy: 0.8597
Epoch 2/20
30/30 [==============================] - 1s 26ms/step - loss: 0.3663 - accuracy: 0.8832 - val_loss: 0.3372 - val_accuracy: 0.8805
Epoch 3/20
30/30 [==============================] - 1s 19ms/step - loss: 0.2752 - accuracy: 0.9077 - val_loss: 0.2982 - val_accuracy: 0.8838
Epoch 4/20
30/30 [==============================] - 0s 16ms/step - loss: 0.2194 - accuracy: 0.9234 - val_loss: 0.2903 - val_accuracy: 0.8836
Epoch 5/20
30/30 [==============================] - 0s 15ms/step - loss: 0.1836 - accuracy: 0.9386 - val_loss: 0.2919 - val_accuracy: 0.8841
Epoch 6/20
30/30 [==============================] - 0s 15ms/step - loss: 0.1596 - accuracy: 0.9462 - val_loss: 0.2787 - val_accuracy: 0.8882
Epoch 7/20
30/30 [==============================] - 0s 15ms/step - loss: 0.1374 - accuracy: 0.9551 - val_loss: 0.2937 - val_accuracy: 0.8841
Epoch 8/20
30

**Practice Model**

We can enhance by deleting the most common words

In [6]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)
common_stop_words=["a","an","and","the","in","of","to","that","this","for"]

def delete_common_stop_words(train_data, common_stop_words):
    word_index = imdb.get_word_index()
    for i in range(len(common_stop_words)):
      if word_index[common_stop_words[i]]:
         common_stop_word_index = word_index[common_stop_words[i]]
         for i in range(len(train_data)):
            for j in range(len(train_data[i])):
              if common_stop_word_index in train_data[i]:
                train_data[i].remove(common_stop_word_index)
    return train_data

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = delete_common_stop_words(train_data, common_stop_words)
train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
Practice_Run  = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 3s 71ms/step - loss: 0.5501 - accuracy: 0.7731 - val_loss: 0.4208 - val_accuracy: 0.8678
Epoch 2/20
30/30 [==============================] - 1s 20ms/step - loss: 0.3467 - accuracy: 0.8894 - val_loss: 0.3318 - val_accuracy: 0.8782
Epoch 3/20
30/30 [==============================] - 0s 17ms/step - loss: 0.2566 - accuracy: 0.9146 - val_loss: 0.2856 - val_accuracy: 0.8898
Epoch 4/20
30/30 [==============================] - 0s 17ms/step - loss: 0.2063 - accuracy: 0.9326 - val_loss: 0.2844 - val_accuracy: 0.8842
Epoch 5/20
30/30 [==============================] - 0s 16ms/step - loss: 0.1697 - accuracy: 0.9437 - val_loss: 0.2764 - val_accuracy: 0.8902
Epoch 6/20
30/30 [==============================] - 0s 16ms/step - loss: 0.1430 - accuracy: 0.9539 - val_loss: 0.2837 - val_accuracy: 0.8883
Epoch 7/20
30/30 [==============================] - 0s 16ms/step - loss: 0.1212 - accuracy: 0.9621 - val_loss: 0.2956 - val_accuracy: 0.8859
Epoch 8/20
30

### Using early stopping

We could using early stopping class in the Keras library to accomplish this step

**Original model**

In [7]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4)

Epoch 1/50
30/30 [==============================] - 3s 65ms/step - loss: 0.5333 - accuracy: 0.7663 - val_loss: 0.4068 - val_accuracy: 0.8519
Epoch 2/50
30/30 [==============================] - 1s 25ms/step - loss: 0.3274 - accuracy: 0.8913 - val_loss: 0.3289 - val_accuracy: 0.8764
Epoch 3/50
30/30 [==============================] - 0s 16ms/step - loss: 0.2402 - accuracy: 0.9219 - val_loss: 0.2977 - val_accuracy: 0.8827
Epoch 4/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1995 - accuracy: 0.9343 - val_loss: 0.2856 - val_accuracy: 0.8840
Epoch 5/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1654 - accuracy: 0.9459 - val_loss: 0.2982 - val_accuracy: 0.8798
Epoch 6/50
30/30 [==============================] - 1s 23ms/step - loss: 0.1427 - accuracy: 0.9538 - val_loss: 0.2853 - val_accuracy: 0.8880
Epoch 7/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1234 - accuracy: 0.9621 - val_loss: 0.2949 - val_accuracy: 0.8854
Epoch 8/50
30

So we could apply the early stopping method as shown below

**Practice**

In [8]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="accuracy",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=0.9,
    restore_best_weights=True,
    start_from_epoch=0,
)
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])


model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_practice = model.fit(train_data, train_labels,
                             epochs=50, batch_size=512, validation_split=0.4, callbacks=[callback])

Epoch 1/50
30/30 [==============================] - 4s 99ms/step - loss: 0.5711 - accuracy: 0.7455 - val_loss: 0.4388 - val_accuracy: 0.8673
Epoch 2/50
30/30 [==============================] - 1s 36ms/step - loss: 0.3578 - accuracy: 0.8913 - val_loss: 0.3280 - val_accuracy: 0.8849
Epoch 3/50
30/30 [==============================] - 1s 19ms/step - loss: 0.2604 - accuracy: 0.9191 - val_loss: 0.3262 - val_accuracy: 0.8675
Epoch 4/50
30/30 [==============================] - 1s 17ms/step - loss: 0.2067 - accuracy: 0.9341 - val_loss: 0.3007 - val_accuracy: 0.8794
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 0.1736 - accuracy: 0.9443 - val_loss: 0.2805 - val_accuracy: 0.8887
Epoch 6/50
30/30 [==============================] - 0s 17ms/step - loss: 0.1451 - accuracy: 0.9554 - val_loss: 0.2861 - val_accuracy: 0.8883
Epoch 7/50
30/30 [==============================] - 0s 16ms/step - loss: 0.1261 - accuracy: 0.9615 - val_loss: 0.2946 - val_accuracy: 0.8867
Epoch 8/50
30

In [9]:
len(history_practice.history['loss'])

37

From the experiments above we could saw that the early stopping take action in the epoch 37 with the accuracy of 0.9999. This is our best results so far and it does save a lot of computing resources

### Regularizing your model

#### Reducing the network's size

**Original model**

In [10]:
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 3s 72ms/step - loss: 0.5575 - accuracy: 0.7585 - val_loss: 0.4352 - val_accuracy: 0.8604
Epoch 2/20
30/30 [==============================] - 1s 20ms/step - loss: 0.3561 - accuracy: 0.8901 - val_loss: 0.3314 - val_accuracy: 0.8809
Epoch 3/20
30/30 [==============================] - 1s 19ms/step - loss: 0.2671 - accuracy: 0.9127 - val_loss: 0.2924 - val_accuracy: 0.8892
Epoch 4/20
30/30 [==============================] - 0s 16ms/step - loss: 0.2160 - accuracy: 0.9282 - val_loss: 0.2783 - val_accuracy: 0.8888
Epoch 5/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1821 - accuracy: 0.9386 - val_loss: 0.2744 - val_accuracy: 0.8915
Epoch 6/20
30/30 [==============================] - 0s 16ms/step - loss: 0.1555 - accuracy: 0.9503 - val_loss: 0.2766 - val_accuracy: 0.8895
Epoch 7/20
30/30 [==============================] - 0s 17ms/step - loss: 0.1359 - accuracy: 0.9563 - val_loss: 0.2931 - val_accuracy: 0.8850
Epoch 8/20
30

**Version of the model with lower capacity**

In [11]:
model = keras.Sequential([
    layers.Dense(4, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_smaller_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 3s 63ms/step - loss: 0.6216 - accuracy: 0.6574 - val_loss: 0.5721 - val_accuracy: 0.8186
Epoch 2/20
30/30 [==============================] - 1s 26ms/step - loss: 0.5160 - accuracy: 0.8071 - val_loss: 0.4987 - val_accuracy: 0.8224
Epoch 3/20
30/30 [==============================] - 1s 20ms/step - loss: 0.4293 - accuracy: 0.8846 - val_loss: 0.4207 - val_accuracy: 0.8720
Epoch 4/20
30/30 [==============================] - 1s 20ms/step - loss: 0.3395 - accuracy: 0.9163 - val_loss: 0.3594 - val_accuracy: 0.8738
Epoch 5/20
30/30 [==============================] - 0s 16ms/step - loss: 0.2692 - accuracy: 0.9321 - val_loss: 0.3183 - val_accuracy: 0.8869
Epoch 6/20
30/30 [==============================] - 0s 16ms/step - loss: 0.2222 - accuracy: 0.9418 - val_loss: 0.2983 - val_accuracy: 0.8874
Epoch 7/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1896 - accuracy: 0.9494 - val_loss: 0.2854 - val_accuracy: 0.8887
Epoch 8/20
30

**Version of the model with higher capacity**

In [12]:
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_larger_model = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 7s 204ms/step - loss: 0.5583 - accuracy: 0.7172 - val_loss: 0.3391 - val_accuracy: 0.8692
Epoch 2/20
30/30 [==============================] - 3s 104ms/step - loss: 0.3138 - accuracy: 0.8699 - val_loss: 0.2842 - val_accuracy: 0.8840
Epoch 3/20
30/30 [==============================] - 3s 101ms/step - loss: 0.2295 - accuracy: 0.9081 - val_loss: 0.3785 - val_accuracy: 0.8491
Epoch 4/20
30/30 [==============================] - 3s 100ms/step - loss: 0.1730 - accuracy: 0.9326 - val_loss: 0.2905 - val_accuracy: 0.8905
Epoch 5/20
30/30 [==============================] - 3s 99ms/step - loss: 0.1452 - accuracy: 0.9464 - val_loss: 0.3415 - val_accuracy: 0.8593
Epoch 6/20
30/30 [==============================] - 3s 102ms/step - loss: 0.0997 - accuracy: 0.9663 - val_loss: 0.3279 - val_accuracy: 0.8878
Epoch 7/20
30/30 [==============================] - 3s 105ms/step - loss: 0.0861 - accuracy: 0.9742 - val_loss: 0.3429 - val_accuracy: 0.8862
Epoch 8

#### Adding weight regularization

**Adding L2 weight regularization to the model**

In [13]:
model = keras.Sequential([
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(16,
                 kernel_regularizer=regularizers.l2(0.002),
                 activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_l2_reg = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 4s 97ms/step - loss: 0.6544 - accuracy: 0.7656 - val_loss: 0.5184 - val_accuracy: 0.8583
Epoch 2/20
30/30 [==============================] - 1s 23ms/step - loss: 0.4437 - accuracy: 0.8843 - val_loss: 0.4404 - val_accuracy: 0.8578
Epoch 3/20
30/30 [==============================] - 1s 18ms/step - loss: 0.3593 - accuracy: 0.9027 - val_loss: 0.3767 - val_accuracy: 0.8852
Epoch 4/20
30/30 [==============================] - 1s 18ms/step - loss: 0.3145 - accuracy: 0.9181 - val_loss: 0.3582 - val_accuracy: 0.8888
Epoch 5/20
30/30 [==============================] - 0s 17ms/step - loss: 0.2865 - accuracy: 0.9263 - val_loss: 0.3549 - val_accuracy: 0.8877
Epoch 6/20
30/30 [==============================] - 1s 21ms/step - loss: 0.2704 - accuracy: 0.9339 - val_loss: 0.3828 - val_accuracy: 0.8735
Epoch 7/20
30/30 [==============================] - 0s 17ms/step - loss: 0.2548 - accuracy: 0.9401 - val_loss: 0.3562 - val_accuracy: 0.8852
Epoch 8/20
30

**Different weight regularizers available in Keras**

In [14]:

regularizers.l1(0.001)
regularizers.l1_l2(l1=0.001, l2=0.001)

#### Adding dropout

**Adding dropout to the IMDB model**

In [15]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(16, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history_dropout = model.fit(
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 [==============================] - 3s 52ms/step - loss: 0.6148 - accuracy: 0.6718 - val_loss: 0.4932 - val_accuracy: 0.8439
Epoch 2/20
30/30 [==============================] - 1s 21ms/step - loss: 0.4808 - accuracy: 0.7973 - val_loss: 0.3767 - val_accuracy: 0.8791
Epoch 3/20
30/30 [==============================] - 1s 17ms/step - loss: 0.3998 - accuracy: 0.8442 - val_loss: 0.3246 - val_accuracy: 0.8815
Epoch 4/20
30/30 [==============================] - 1s 18ms/step - loss: 0.3397 - accuracy: 0.8810 - val_loss: 0.2944 - val_accuracy: 0.8875
Epoch 5/20
30/30 [==============================] - 1s 22ms/step - loss: 0.2965 - accuracy: 0.8998 - val_loss: 0.2787 - val_accuracy: 0.8897
Epoch 6/20
30/30 [==============================] - 1s 20ms/step - loss: 0.2629 - accuracy: 0.9143 - val_loss: 0.2718 - val_accuracy: 0.8922
Epoch 7/20
30/30 [==============================] - 1s 17ms/step - loss: 0.2282 - accuracy: 0.9248 - val_loss: 0.3010 - val_accuracy: 0.8812
Epoch 8/20
30

## Summary